In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [2]:
class_num = 13
batch_size=32
target_size=(200,200)

gen = ImageDataGenerator(rescale=1/255.)
train_gen = gen.flow_from_directory('./train/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=True)
valid_gen = gen.flow_from_directory('./valid/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=True)
test_gen = gen.flow_from_directory('./test/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=True)

Found 39780 images belonging to 13 classes.
Found 13260 images belonging to 13 classes.
Found 13260 images belonging to 13 classes.


In [3]:
InceptionResNetV2_base_loaded = tf.keras.models.load_model('./save_models/InceptionResNetV2_13class')
EffNetV2M_base_loaded = tf.keras.models.load_model('./save_models/EffNetV2M_13class')
VGG16_base_loaded = tf.keras.models.load_model('./save_models/VGG16_13class')   

In [4]:
modelacc_InceptionResNetV2 = 0.87
modelacc_EffNetV2M = 0.91
modelacc_VGG16 = 0.90

In [7]:
predcount = 0
hitcount = 0
forcount = 0
accuracy = 0

for image in test_gen:
    forcount += 1
    # predlist = InceptionResNetV2_base_loaded.predict(image)
    
    pred_InceptionResNetV2 = InceptionResNetV2_base_loaded.predict(image)
    pred_EffNetV2M = EffNetV2M_base_loaded.predict(image)
    pred_VGG16 = VGG16_base_loaded.predict(image)
    
    for i in range(len(pred_InceptionResNetV2)):
        # if np.argmax(predlist[i]) == np.argmax(image[1][i]) : hitcount += 1
        # predcount += 1
        
        votelist = [0 for x in range(class_num)]
        votelist[np.argmax(pred_InceptionResNetV2[i])] += 1*modelacc_InceptionResNetV2
        votelist[np.argmax(pred_EffNetV2M[i])] += 1*modelacc_EffNetV2M
        votelist[np.argmax(pred_VGG16[i])] += 1*modelacc_VGG16
        
        if np.argmax(votelist) == np.argmax(image[1][i]) : hitcount += 1
        predcount += 1        
        
        
        
    if forcount%20 == 0 : print(forcount, ':', hitcount / predcount * 100)
    if forcount%300 == 0 : break
accuracy = hitcount / predcount * 100
accuracy

20 : 91.09375
40 : 92.1875
60 : 92.91666666666667
80 : 92.96875
100 : 93.15625
120 : 92.76041666666667
140 : 92.8125
160 : 92.96875
180 : 92.89930555555556
200 : 92.875
220 : 92.9971590909091
240 : 93.05989583333333
260 : 93.17307692307692
280 : 93.24776785714286
300 : 93.25


93.25